In [168]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from pathlib import Path
import time
from selenium.webdriver.common.action_chains import ActionChains

In [217]:
WHITNEY_URL = 'https://www.recreation.gov/permits/233260'


def get_whitney_permit_message_2021():
    """Gets the status of whitney permits, 2021."""

    driver = webdriver.Firefox()
    driver.get(WHITNEY_URL)

    def _input_parameters_and_parse_availability(permit_type: str,
                                                 num_people: int = 1):
        """Inputs the parameters the NPS website wants."""
        # There's a dropdown for {Overnight, Day Use}
        # Overnight is index 1
        # Day use is index 2

        permit_type_map = {"dayuse": 2, "overnight": 1}
        try:
            selection_index = permit_type_map[permit_type]
        except KeyError:
            raise ValueError(f"Expected {list(permit_type_map.keys())}")

        select = Select(
            driver.find_element_by_xpath("//select[@id='division-selection']"))
        select.select_by_index(selection_index)
        time.sleep(2.0)

        # Select the number of people in your group.
        your_input = driver.find_element_by_xpath(
            "//input[@id='number-input-']")
        your_input.clear()
        your_input.send_keys(f"{num_people}")

        # The table cells with nonzero aria-label corresponds to the "calendar" view
        # showing available permits.
        table_cells = [
            el for el in driver.find_elements_by_xpath("//td")
            if el.get_attribute('aria-label') is not None
        ]

        # Get rid of anything that's None or "Not Available".
        available = [
            el for el in table_cells
            if not 'Not available' in el.get_attribute('aria-label')
        ]

        # For all the rest, hover over to see the number of permits
        res = {}
        for a in available:
            date = a.get_attribute('aria-label')
            # Scroll to the thing
            x, y = a.location['x'], a.location['y']
            scroll_by_coord = 'window.scrollTo(%s,%s);' % (x, y)
            scroll_nav_out_of_way = 'window.scrollBy(0, -120);'
            driver.execute_script(scroll_by_coord)
            driver.execute_script(scroll_nav_out_of_way)

            ac = ActionChains(driver)
            ac.move_to_element(a)
            ac.perform()
            time.sleep(0.5)
            text = driver.find_element_by_xpath(
                "//div[contains(text(),'People:')]").text
            res[date] = text

        return res

    dayres = _input_parameters_and_parse_availability("dayuse", 1)
    overres = _input_parameters_and_parse_availability("overnight", 1)
    driver.close()

    # Turn this into a nice message
    output = "Day Use Permits\n"
    output += "===============\n"
    for date, lol in dayres.items():
        output += f"{date} - {lol}\n"

    output += "\n\n"

    output += "Overnight Permits\n"
    output += "=================\n"
    for date, lol in overres.items():
        output += f"{date} - {lol}\n"

    output += "\n\n"
    return output

In [218]:
last_fetch = 0
last_message = None

while True:
    if time.time() - last_fetch < 60:
        continue
        
    try:
        message = get_whitney_permit_message_2021()
        print(message)
        last_message = message
        
        if message != last_message:
            print("Message changed!")

    except Exception as err:
        print(err)

    last_fetch = time.time()

Day Use Permits
Friday, October 1, 2021 - People: 1 out of 99+
Friday, October 8, 2021 - People: 1 out of 99+
Thursday, October 14, 2021 - People: 1 out of 99+


Overnight Permits
Friday, October 8, 2021 - People: 2 out of 60



Day Use Permits
Friday, October 1, 2021 - People: 1 out of 99+
Friday, October 8, 2021 - People: 1 out of 99+
Thursday, October 14, 2021 - People: 1 out of 99+


Overnight Permits



Day Use Permits
Friday, October 1, 2021 - People: 1 out of 99+
Friday, October 8, 2021 - People: 1 out of 99+
Thursday, October 14, 2021 - People: 1 out of 99+


Overnight Permits



Day Use Permits
Friday, October 1, 2021 - People: 1 out of 99+
Friday, October 8, 2021 - People: 1 out of 99+
Thursday, October 14, 2021 - People: 1 out of 99+


Overnight Permits



Day Use Permits
Friday, October 1, 2021 - People: 1 out of 99+
Friday, October 8, 2021 - People: 1 out of 99+
Thursday, October 14, 2021 - People: 1 out of 99+


Overnight Permits



Day Use Permits
Friday, October 1, 2021

KeyboardInterrupt: 

In [ ]:
start_time = time.time()

In [212]:
time.time() - start_time

11.110034942626953

In [213]:
time.time()

1633117533.035893